In [29]:
import pandas as pd
df_org = pd.read_csv("./Datasets/galaxy_users.csv")
df_org.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [49]:
df_org.shape

(7032, 21)

In [36]:
df_org.columns 

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [47]:
cols=['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
df1=df_org.copy()
df1=df1[df1[cols].apply(lambda s: s.str.contains("Yes|No")).sum(axis=1)==6]
df1=df1.assign(num_service_add=df1[cols].apply(lambda s: s.str.contains("Yes")).sum(axis=1))
s1=df1.groupby('num_service_add')['TechSupport'].count()[1]
s6=df1.groupby('num_service_add')['TechSupport'].count()[6]
A1 = round(s1/s6,1)
A1

3.4

In [54]:
df1=df1[df1[cols].apply(lambda s: s!="No internet service")]
df1.shape()


TypeError: 'tuple' object is not callable

In [53]:
df1[cols].apply(lambda s: s.str.contains("Yes")).sum(axis=1)
df1.shape

(7032, 22)

In [ ]:
df1.head(10)

In [35]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [40]:
cols=['tenure','MonthlyCharges',"MonthUsing"]
df2=df_org[['tenure','MonthlyCharges']]
df2['MonthUsing']=df_org['TotalCharges']//df_org['MonthlyCharges']
df2.head()

<ipython-input-40-fae5c334f2de>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['MonthUsing']=df_org['TotalCharges']//df_org['MonthlyCharges']


,tenure,MonthlyCharges,MonthUsing
0,1,29.85,1.0
1,34,56.95,33.0
2,2,53.85,2.0
3,45,42.30,43.0
4,2,70.70,2.0


In [56]:
df2.corr(method="pearson")['tenure']

tenure            1.000000
MonthlyCharges    0.246862
MonthUsing        0.998831
Name: tenure, dtype: float64

In [43]:
df_org.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [63]:
df2=df_org.copy()
cols=['SeniorCitizen', 'Partner', 'Dependents','tenure','MonthlyCharges','TotalCharges',
      'OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingMovies','PaperlessBilling']
cols_on_off=['SeniorCitizen', 'Partner', 'Dependents','OnlineSecurity','OnlineBackup',
             'DeviceProtection','TechSupport','StreamingMovies','PaperlessBilling']
df2[cols_on_off]=df2[cols_on_off].applymap(lambda x : 1 if x=="Yes" else (0 if x=="No" else -1))
df2.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,-1,1,0,1,No,No phone service,DSL,0,...,0,0,No,0,Month-to-month,1,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,-1,0,0,34,Yes,No,DSL,1,...,1,0,No,0,One year,0,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,-1,0,0,2,Yes,No,DSL,1,...,0,0,No,0,Month-to-month,1,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,-1,0,0,45,No,No phone service,DSL,1,...,1,1,No,0,One year,0,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,-1,0,0,2,Yes,No,Fiber optic,0,...,0,0,No,0,Month-to-month,1,Electronic check,70.70,151.65,Yes
